# Helper functions

In [47]:
#####################################################################################
#                                   Importing Libraries                             #
#####################################################################################
from dotenv import load_dotenv, find_dotenv
import os
import pandas as pd
import requests
from collections import defaultdict
from pathlib import Path
import typing
from typing import List, Dict

#####################################################################################
#                                   Custom Type Hints                               #
#####################################################################################

Filepath = typing.Union[str, bytes, os.PathLike]

#####################################################################################
#                                   Ad hoc functions                                #
#####################################################################################

def debug_token(token: str) -> str:

    payload = {"input_token": token}

    base_url = "https://graph.facebook.com"
    extension = f"/v10.0/debug_token?input_token={token}"

    url = base_url + extension

    res = requests.get(url, data=payload).json()

    return res

# Finds published posts for a given page
def find_published_posts(page_token: str = None, page_id: str = None) -> List:
    if page_token is None:
        page_token = find_page_token()

    if page_id is None:
        page_id = find_page_id()

    base_url = "https://graph.facebook.com"
    add_on_url = f"/v10.0/{page_id}/published_posts"
    url = base_url + add_on_url

    params = {"access_token": page_token}

    res = requests.get(url, params=params)

    d = res.json()

    post_ids = [ele["id"] for ele in d["data"]]

    return post_ids


##################################################################################
#              Use these functions to create variables from the environment      #  
##################################################################################

def find_user_token() -> str:
    load_dotenv(find_dotenv())

    user_token = os.getenv("user_token")
    return user_token


def find_user_id() -> str:
    load_dotenv(find_dotenv())

    user_id = os.getenv("user_id")
    return user_id


def find_page_id() -> str:
    load_dotenv(find_dotenv())

    page_id = os.getenv("page_id")
    return page_id


# This is used to define a page token
def find_page_token(page_id: str = None, user_token: str = None) -> str:
    if page_id is None:
        page_id = find_page_id()
    if user_token is None:
        user_token = find_user_token()

    page_token_url = (
        f"https://graph.facebook.com/{page_id}?fields=access_token&access_token={user_token}"
    )
    
    res = requests.get(page_token_url)
    res_json = res.json()

    try:
        page_token = res_json['access_token']
    except KeyError:
        print("Error loading using user token - using saved page token")
        page_token = os.getenv("page_token")
    return page_token



###########################################################################################
#                                 Primary Logic                                           #
###########################################################################################


def read_metrics(path: Filepath) -> dict:
    """Reads the metrics from a spreadsheet in the current directory containing Metric Name and Valid Period columns"""
    
    # Read in the specified spreadsheet
    metric_spreadsheet = pd.read_excel(path)
    

    # Initialise a dictionary to hold the metrics and their valid periods
    metrics = {}

    for index, row in metric_spreadsheet.iterrows():
        # Extract the metric name and valid periods from eaech row of the tabl
        metric_name = row['Metric Name']
        valid_periods = row['Valid Periods'].split(', ')

        # Remove spaces and asterisks from metric names
        metric_name = metric_name.replace(" ", "")
        metric_name = metric_name.replace("*", "")

        # Create a new dictionary entry with the valid values
        # metric name: list of valid periods
        metrics[metric_name] = valid_periods
        
    return metrics


# Returns the values for a given page and requested metrics
def request_metrics(list_of_metrics: List[str], 
                    page_id: str = None, 
                    page_token: str = None, 
                    period: str = None) -> Dict:
    """
    Give me a list of metrics and a page token and I'll return the value (dated at the most recent time)
    Output can be given straight to pandas to pd.Dataframe.from_dict --> df.to_excel
    """
    
    # Define variables if not defined by the caller
    if page_token is None:
        page_token = find_page_token()
        
    if page_id is None:
        page_id = find_page_id()

        
    # Create the URL to send a request to    
    base_url = "https://graph.facebook.com"
    page_id = find_page_id()

    url = create_request_url(list_of_metrics, page_id)

    
    # Additional parameters to be included in the request
    params = {
        "access_token": page_token,
        "show_description_from_api_doc": "true",
    }
    
    valid_periods = {"day", "week", "days_28", "month", "lifetime"}
    if period in valid_periods:
        params["period"] = period
    else:
        print("Period invalid or not found")

    
    # Send a request for the metrics
    res = requests.get(url, params=params)

    # Return the json object
    d = res.json()

    return d

# Takes a list of metrics and returns the URL to send a request to
def create_request_url(list_of_metrics: List[str], page_id: str = None) -> str:
    
    # Define variables if not set by the caller
    if page_id is None:
        page_id = find_page_id()
        
        
    # Base URL should be invariant
    BASE_URL = 'https://graph.facebook.com'
    
    # The metrics requested must be joined by a ','
    metric_string = ','.join(list_of_metrics)
    
    # Create URL structure as defined by the API
    # https://developers.facebook.com/docs/graph-api/reference/v11.0/insights
    multiple_request_url = f"{BASE_URL}/v10.0/{page_id}/insights?metric={metric_string}"
    
    return multiple_request_url


# Processes the data returned from request_metrics()
def data_to_dict(d: dict) -> Dict:
    """This should take in the DATA from a request (this can be done with res.json()['data'])
    And it will output a defaultdict of defaultdicts with the structure
    date --> metric --> value
    """
    output_d = defaultdict(defaultdict)

    # This is how the output_d will be structured when it's returned
    # output_d['date']['metric'] = value

    # Iterating through each metric that we're given
    for ele in d:
        variable_name = ele["name"]

        # Iterating through each value for a specific metric
        for values in ele["values"]:
            # There should be a list of values, each for a different date
            # Structure of this is a list of dicts
            # We should turn metric -> date -> value into date -> metric -> value

            # We wrap this in a try because some metrics don't come as integers (they could be a list or a dict)
            try:
                if isinstance(values["value"], int) == True:
                    date = values["end_time"]
                    value = values["value"]
                    metric_name = variable_name

                    output_d[date][metric_name] = value
            except Exception:
                continue

    return output_d


# Outputs a dictionary as a spreadsheet
def dict_to_spreadsheet(d: Dict, file_name: str = "output.xlsx", sheet_name: str = None):
    """This takes in a dict and outputs the spreadsheet to the given filename"""
    # Parameter for index should be removed in df.to_excel
    df = pd.DataFrame.from_dict(d)
    
    if sheet_name is not None:
        df.to_excel(file_name, sheet_name = sheet_name)
    else:
        df.to_excel(file_name)

# Setting variables

In [5]:
page_token = find_page_token()
page_id = find_page_id()

BASE_URL = 'https://graph.facebook.com'

# Establishing Metrics to be searched

In [ ]:
# Read metrics spreadsheet
metrics = read_metrics(Path("columns.xlsx"))

In [22]:
# We want to create an inverted dictionary with keys being each period
# period: metric_name
metrics_inverted = defaultdict(list)
for metric, period_list in metrics.items():
    for period in period_list:
        metrics_inverted[period].append(metric)
        
# Now we make requests going back as far as we can using each period type

# Requesting the defined metric (for the most recent date)

In [70]:
# Create data structure to contain all the data we want in our spreadsheetj

# period: 
# {date: data}
dict_of_metrics = dict()


# Create a dictionary for each "period"
for period, metrics in metrics_inverted.items():
    # Create requests for that kind of metric
    result = request_metrics(metrics, period = period)
    
    # Create the dictionary to store our data
    working_dict = data_to_dict(result['data'])
    
    # Traverse the graph for previous dates and build on the current dictionary
    
    # Extend the current dictionary by traversing gthe graph

    for i in range(500):
        # result should start as a dictionary

        # Extract the URL to find the previous event in the graph
        previous = result['paging']['previous']

        # Traverse to the previous node and save the request to result
        result = requests.get(previous).json()

        # Extend the dictionary with the metrics for the previous date
        working_dict = {**working_dict, **data_to_dict(result['data'])}
    
    dict_of_metrics[period] = working_dict

# Write our data to a spreadsheet

In [73]:
writer = pd.ExcelWriter("testOutput.xlsx",
                       engine = "xlsxwriter")

# Write each metric to its own sheet in the spreadsheet
for period, metrics in dict_of_metrics.items():
    df = pd.DataFrame.from_dict(metrics)
    
    df.to_excel(writer, sheet_name = period)

writer.save()